In [1]:
from pyper.viper_interface.viper_classes import PolhemusViper
from pyper.io.io_utils import extract_data_from_frame_continuous
from threading import Event
from multiprocessing.pool import ThreadPool
import time
import pandas as pd

In [2]:
viper = PolhemusViper()
dev, ep = viper.connect(return_endpoint=True)

In [4]:
stop_con = Event()
pool = ThreadPool(processes=1)
async_result = pool.apply_async(viper.read_continuous, (dev, ep, stop_con))

time.sleep(5)


stop_con.set()
df = async_result.get()
# df = Thread(target=viper.read_continuous, args=(dev, ep, stop_con), daemon=True).start()

In [6]:
df1 = pd.DataFrame()
for i in range(len(df)):
    df2 = extract_data_from_frame_continuous(df[i])
    df1 = pd.concat([df1, df2], ignore_index=True)
df1

,sensor_n,frames,time,x,y,z,azimuth,elevation,roll
0,1,9,2022-05-06T09:17:45.334983,9.762692,7.961395,1.993608,151.363281,-20.453165,-129.196365
1,2,9,2022-05-06T09:17:45.334983,45.484043,-35.102631,-0.760467,123.036224,-3.182949,129.367310
2,3,9,2022-05-06T09:17:45.334983,4.096188,-18.099319,-2.013289,-124.473633,-6.801953,-63.929100
3,1,1617,2022-05-06T09:17:45.337980,9.761791,7.960741,1.994194,151.372131,-20.456442,-129.203354
4,2,1617,2022-05-06T09:17:45.337980,45.481457,-35.104912,-0.783368,123.034592,-3.239926,129.388412
...,...,...,...,...,...,...,...,...,...
1804,2,2817,2022-05-06T09:17:50.337941,45.475395,-35.108097,-0.774676,123.022598,-3.218532,129.354904
1805,3,2817,2022-05-06T09:17:50.337941,4.097096,-18.098703,-2.009993,-124.480560,-6.783955,-63.904408
1806,1,2819,2022-05-06T09:17:50.346570,9.761262,7.960866,1.994661,151.375076,-20.453459,-129.199387
1807,2,2819,2022-05-06T09:17:50.346570,45.473671,-35.110291,-0.770494,123.013023,-3.206393,129.353226


In [7]:
df1[40:100]

,sensor_n,frames,time,x,y,z,azimuth,elevation,roll
40,2,1641,2022-05-06T09:17:45.438216,45.478527,-35.104279,-0.775633,123.039185,-3.218620,129.372879
41,3,1641,2022-05-06T09:17:45.438216,4.096465,-18.098759,-2.012087,-124.476341,-6.794476,-63.920429
42,1,1643,2022-05-06T09:17:45.446728,9.761788,7.960829,1.993987,151.370590,-20.454918,-129.203293
43,2,1643,2022-05-06T09:17:45.446728,45.480549,-35.102142,-0.777918,123.046204,-3.224627,129.375427
44,3,1643,2022-05-06T09:17:45.446728,4.096444,-18.098753,-2.012063,-124.476395,-6.794622,-63.920364
45,1,1645,2022-05-06T09:17:45.454870,9.761839,7.960783,1.994096,151.370483,-20.456161,-129.203934
46,2,1645,2022-05-06T09:17:45.454870,45.479774,-35.104465,-0.772332,123.041618,-3.210968,129.368973
47,3,1645,2022-05-06T09:17:45.454870,4.096455,-18.098736,-2.012056,-124.476364,-6.794526,-63.920296
48,1,1647,2022-05-06T09:17:45.462974,9.761864,7.960727,1.994233,151.370819,-20.457525,-129.204819
49,2,1647,2022-05-06T09:17:45.462974,45.479145,-35.105602,-0.772203,123.035698,-3.207735,129.370010


In [3]:
viper.get_single_pno(dev=dev, ep=ep, pno_mode="standard")

error: unpack requires a buffer of 4 bytes

In [3]:
viper.start_continuous(dev, ep, pno_mode="standard", framecount="reset")

[86, 80, 82, 67, 24, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 103, 246, 0, 0]


In [4]:
dev.read(ep, 100, 200)

array('B', [86, 80, 82, 80, 84, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 66, 0, 0, 101, 231, 15, 65, 139, 200, 180, 64, 106, 101, 20, 64, 105, 254, 43, 67, 7, 125, 218, 193, 119, 110, 35, 195, 0, 0, 0, 0, 1, 130, 4, 0, 10, 166, 51, 66, 127, 109, 7, 194, 64, 72, 113, 62, 40, 135, 238, 66, 174, 8, 212, 192, 192, 81, 3, 67, 0, 0, 0, 0, 34, 165, 0, 0])

In [3]:
df = viper.get_single_pno(dev=dev, ep=ep, pno_mode="standard")
df

,sensor_n,frames,time,x,y,z,azimuth,elevation,roll
0,1,24154,2022-05-05T22:48:39.533252,9.010628,5.649452,2.351197,172.175293,-28.035522,-165.601578
1,2,24154,2022-05-05T22:48:39.533252,44.924061,-33.885765,0.258443,119.227028,-6.605873,131.344360
